# Text Generation


In [2]:
import os
import pickle

import numpy as np
import tensorflow

import time

## Helper functions

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Here, we implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  We create a dictionary for the symbols, where the symbol is the key and value is the token.

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. We don't use a token that could be confused as a word. Instead of using the token "dash", we use therfore use "||dash||".

In [3]:
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data

def token_lookup():
    """
    Generates a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokenize = {
        '.': '||period||',
        ',': '||comma||',
        '"': '||quotation_mark||',
        ';': '||semicolon||',
        '!': '||exclamation_mark||',
        '?': '||question_mark||',
        '(': '||left_parentheses||',
        ')': '||right_parentheses||',
        '--':'||dash||',
        '\n':'||return||'
    }
    return tokenize

def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocesses Text Data
    """
    text = load_data(dataset_path)
    
    # Ignore notice, since we don't use it for analysing the data
    text = text[81:]

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text)
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))


def load_preprocess():
    """
    Loads the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))


def save_params(params):
    """
    Saves parameters to file
    """
    pickle.dump(params, open('params.p', 'wb'))


def load_params():
    """
    Loads parameters from file
    """
    return pickle.load(open('params.p', mode='rb'))



## Implement Preprocessing Functions


### Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function, we create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`


In [8]:
def create_lookup_tables(text):
    """
    Creates lookup tables for vocabulary
    :param text: The text split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab

## Get the Data



In [9]:
data_dir = './data/anna.txt'
text = load_data(data_dir)

## Explore the Data
We can play around with `view_sentence_range` to view different parts of the data.

In [11]:
view_sentence_range = (5000, 5200)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 29230

The sentences 5000 to 5200:
settled herself comfortably. An invalid lady had already lain down to
sleep. Two other ladies began talking to Anna, and a stout elderly lady
tucked up her feet, and made observations about the heating of the
train. Anna answered a few words, but not foreseeing any entertainment
from the conversation, she asked Annushka to get a lamp, hooked it onto
the arm of her seat, and took from her bag a paper knife and an English
novel. At first her reading made no progress. The fuss and bustle were
disturbing; then when the train had started, she could not help
listening to the noises; then the snow beating on the left window and
sticking to the pane, and the sight of the muffled guard passing by,
covered with snow on one side, and the conversations about the terrible
snowstorm raging outside, distracted her attention. Farther on, it was
continually the same again and again: the same shaking and rattling, the
s

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is the first checkpoint.

In [7]:
int_text, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))

In [8]:
len(int_text)

466523

## Build the Neural Network



### Check the Version of TensorFlow and Access to GPU

In [9]:

from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input

In [10]:
def get_inputs():
    """
    Creates TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    
    targets = tf.placeholder(tf.int32,[None, None], name='targets')
    
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return (inputs, targets, learning_rate)


### Build RNN Cell and Initialize
We stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).

- We initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
- We apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)


In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """

    num_layers = 2

    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)

    #drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    cell = tf.contrib.rnn.MultiRNNCell([lstm])

    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    
    
    return (cell, initial_state)

### Word Embedding


In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Creates embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


### Build RNN
We created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
- Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

In [13]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    state = tf.identity(state, name='final_state')
    
    return (outputs, state)

### Build the Neural Network
We use the functions implemented above to:
- Apply embedding to `input_data` using the `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and our `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

In [14]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function

    embed = get_embed(input_data, vocab_size, embed_dim)

    outputs, state = build_rnn(cell, embed)

    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)

    return (logits, state)

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

(Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.)

In [15]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: A list where each item is a tuple of (batch of input, batch of target).
    """
    n_batches = len(int_text) // (batch_size * seq_length)

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    
    ydata[-1] = xdata[0]

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(x_batches, y_batches)))

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [25]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 363

save_dir = './save'

### Build the Graph
Build the graph using the neural network we implemented.

In [26]:

from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(logits,
                                 targets,
                                 tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [ ]:

batches = get_batches(int_text, batch_size, seq_length)

start_time = time.time()

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                elapsed_time = (time.time() - start_time) / 60
                print("".format())
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}   Time elapsed: {:.1f}min'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss,
                    elapsed_time))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')


Epoch   0 Batch    0/121   train_loss = 9.571   Time elapsed: 0.0min

Epoch   3 Batch    0/121   train_loss = 4.983   Time elapsed: 1.6min

Epoch   6 Batch    0/121   train_loss = 4.508   Time elapsed: 3.2min

Epoch   9 Batch    0/121   train_loss = 4.259   Time elapsed: 4.8min

Epoch  12 Batch    0/121   train_loss = 4.061   Time elapsed: 6.4min

Epoch  15 Batch    0/121   train_loss = 3.888   Time elapsed: 8.0min

Epoch  18 Batch    0/121   train_loss = 3.724   Time elapsed: 9.6min

Epoch  21 Batch    0/121   train_loss = 3.573   Time elapsed: 11.2min

Epoch  24 Batch    0/121   train_loss = 3.436   Time elapsed: 12.8min

Epoch  27 Batch    0/121   train_loss = 3.299   Time elapsed: 14.4min

Epoch  30 Batch    0/121   train_loss = 3.174   Time elapsed: 16.0min

Epoch  33 Batch    0/121   train_loss = 3.044   Time elapsed: 17.6min

Epoch  36 Batch    0/121   train_loss = 2.922   Time elapsed: 19.2min

Epoch  39 Batch    0/121   train_loss = 2.799   Time elapsed: 20.8min

Epoch  42 Ba

## Save Parameters


In [28]:
# Save parameters for checkpoint
save_params((seq_length, save_dir))

# Checkpoint

In [4]:

import tensorflow as tf
import numpy as np

_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
seq_length, load_dir = load_params()

## Implement Generate Functions
### Get Tensors
We get the tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  We use the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [5]:
def get_tensors(loaded_graph):
    """
    Gets input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """    
    InputTensor = tf.Graph.get_tensor_by_name(loaded_graph, name='input:0')
    InitialStateTensor = tf.Graph.get_tensor_by_name(loaded_graph, name='initial_state:0')
    FinalStateTensor = tf.Graph.get_tensor_by_name(loaded_graph, name='final_state:0')
    ProbsTensor = tf.Graph.get_tensor_by_name(loaded_graph, name='probs:0')
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [6]:
def pick_word(probabilities, int_to_vocab):
    """
    Picks the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return np.random.choice(list(int_to_vocab.values()), p=probabilities)

## Generate new Text
This will generate the new text. We can set `gen_length` to the length of text we want to generate.

In [40]:
gen_length = 200

prime_word = 'all'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    new_text = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        new_text = new_text.replace(' ' + token.lower(), key)
    new_text = new_text.replace('\n ', '\n')
    new_text = new_text.replace('( ', '(')
        
    print(new_text)

all about anna.

sviazhsky questioned him too that day, she had scarcely had better
quite free in moscow, as though she had been
thinking of nothing but felt his whole figure and felt to blame. he had
made an answer and pain, and for her part she had a great deal
of her acquaintances, her lips was going out without leaving time to the sound
opposite her, and began to move themselves, with her that
sister-in-law was not to sign the baby, to her mother's coldness in his morning,
and he felt with difficulty about this. the other, as an english put
in her having balls, gone up in taking a cup of tea, with
bare fingers, so that he would never allow the colonel.

" disagreeable division of women? be guided by its humiliation, and this is
perhaps in the public domain print."

though it was enough for kitty in his usual, and while he was in
both the minutes above as
